In [1]:
# Download the files
!wget https://github.com/lacykaltgr/continual-learning-ait/archive/refs/heads/main.zip
!unzip main.zip
!find continual-learning-ait-main -type f ! -name "main.ipynb" ! -name "main.py" -exec cp {} . \;

--2023-04-10 23:30:49--  https://github.com/lacykaltgr/continual-learning-ait/archive/refs/heads/main.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/lacykaltgr/continual-learning-ait/zip/refs/heads/main [following]
--2023-04-10 23:30:49--  https://codeload.github.com/lacykaltgr/continual-learning-ait/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 20.27.177.114
Connecting to codeload.github.com (codeload.github.com)|20.27.177.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [ <=>                ] 148.73K   990KB/s    in 0.2s    

2023-04-10 23:30:50 (990 KB/s) - ‘main.zip’ saved [152304]

Archive:  main.zip
f1d2d6e8eb8de0cbe921f1b3945057ba0ef0d509
   creating: continual-learning-ait-main/
  

In [2]:
import tensorflow as tf
from data_preparation import load_dataset
from data_preparation import CLDataLoader

In [3]:
dpt_train, dpt_test = load_dataset('cifar-100')

169001437/169001437 [==============================] - 11s 0us/step


/content/data_preparation.py:60: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array([X_task_train, y_task_train]))
/content/data_preparation.py:68: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array([X_task_test, y_task_test]))


In [8]:
dpt_train[0]

array([[array([[[-0.6627451 , -0.61568627, -0.63921569],
                [-0.74901961, -0.68627451, -0.73333333],
                [-0.31764706, -0.30196078, -0.63137255],
                ...,
                [-0.41960784, -0.34117647, -0.62352941],
                [-0.6627451 , -0.6       , -0.76470588],
                [-0.69411765, -0.64705882, -0.74117647]],

               [[-0.65490196, -0.63137255, -0.6       ],
                [-0.70196078, -0.67058824, -0.67058824],
                [-0.5372549 , -0.52156863, -0.74901961],
                ...,
                [-0.48235294, -0.41960784, -0.63137255],
                [-0.68627451, -0.63137255, -0.75686275],
                [-0.68627451, -0.63921569, -0.73333333]],

               [[-0.64705882, -0.64705882, -0.60784314],
                [-0.6627451 , -0.6627451 , -0.65490196],
                [-0.65490196, -0.63137255, -0.76470588],
                ...,
                [-0.6       , -0.54509804, -0.68627451],
                [-0.6

In [9]:
loader = CLDataLoader(dpt_train, 100, train=True)

ValueError: ignored

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

fig = plt.figure(figsize=(4., 4.))
grid = ImageGrid(fig, 111,
                 nrows_ncols=(2, 2),
                 axes_pad=0.1, )

for ax, i in zip(grid, range(4)):
    ax.imshow(X_train_cifar_100[i] * 0.5 + 0.5)
plt.show()